In [2]:
import pandas as pd
import datetime as dt
import numpy as np
import pm4py
from sklearn.preprocessing import MinMaxScaler

from FU import *

df=pd.read_csv('./rawdata/BPI2016_Clicks_Logged_In.csv', sep=';', encoding='latin-1', keep_default_na=False)



df['time'] = pd.to_datetime(df['TIMESTAMP'])
df['dates'] = df['time'].dt.date

!pip install pm4py

In [3]:

log = pm4py.read_xes('./rawdata/BPI Challenge 2017.xes.gz')
pdf = pm4py.convert_to_dataframe(log)

parsing log, completed traces ::   0%|          | 0/31509 [00:00<?, ?it/s]

In [4]:
# pdf['case:concept:name'].unique()[:25]

In [5]:
# pdf[pdf['case:concept:name']=='Application_1746793196']

#Application_546206358 - denied and refused O_Returned/O_Refused
#Application_1529124572 - accepted O_Accepted
#Application_1085880569 - cancelled  O_Cancelled

In [6]:
case_id_f = 'case:concept:name'
timestamp_f = 'time:timestamp'
activity_f =  'concept:name'

In [7]:
print("Unique traces count", len(pdf['case:concept:name'].unique()))

31509


In [10]:
print("Total events count",len(pdf[activity_f]))

Total events count 1202267


In [9]:
print("Unique events count",len(pdf[activity_f].unique()))

26


In [11]:
print(len(pdf[activity_f])/len(pdf['case:concept:name'].unique()))

38.15630454790695


In [7]:
offer_returned = list(pdf[pdf['concept:name']=='O_Returned']['case:concept:name'].unique())
offer_refused = list(pdf[pdf['concept:name']=='O_Refused']['case:concept:name'].unique())
offer_accepted = list(pdf[pdf['concept:name']=='O_Accepted']['case:concept:name'].unique())
offer_cancelled = list(pdf[pdf['concept:name']=='O_Cancelled']['case:concept:name'].unique())
case_ids = list(pdf['case:concept:name'].unique())

print(len(offer_returned),len(offer_refused),len(offer_accepted),len(offer_cancelled))

21768 3720 17228 15682


In [8]:
case_flags = pd.DataFrame({'CaseID': case_ids})
case_flags['Flag'] = np.where(case_flags['CaseID'].isin(offer_returned), 1, 0)

In [9]:
case_flags.Flag.sum()/case_flags.shape[0]

0.6908502332666857

In [10]:
# print(len(offer_returned),len(offer_refused),len(offer_accepted),len(offer_cancelled))

In [11]:
pdf.head(15)

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,statechange,User_1,A_Concept,Application,ApplState_642383566,complete,2016-01-01 09:52:36.413000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Obtained,User_17,W_Complete application,Workflow,Workitem_1875340971,start,2016-01-02 10:45:22.429000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Released,User_17,W_Complete application,Workflow,Workitem_1452291795,suspend,2016-01-02 10:49:28.816000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,statechange,User_52,A_Accepted,Application,ApplState_99568828,complete,2016-01-02 11:23:04.299000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Created,User_52,O_Create Offer,Offer,Offer_148581083,complete,2016-01-02 11:29:03.994000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,20000.0,44.0,True,498.29,True,979.0,20000.0,NaN


id = 'Application_1691306052'
case_level = pdf[pdf['case:concept:name']==id]
case_level.loc[case_level['concept:name'] == 'O_Accepted']

case_level.index.tolist().index(case_level.loc[case_level['concept:name'] == 'O_Accepted'].index.tolist()[0])

id = 'Application_1746793196'
case_level = pdf[pdf['case:concept:name']==id]

if id in offer_returned:
    outcome_check = 'O_Returned'
if id in offer_accepted:
    outcome_check = 'O_Accepted'
elif id in offer_cancelled:
    outcome_check = 'O_Cancelled'
elif id in offer_refused:
    outcome_check = 'O_Refused'
print(outcome_check)


i=case_level.loc[case_level['concept:name'] == outcome_check].index.tolist()

case_level.index.tolist().index(i[0])

import random

id = case_ids[1]
case_level = pdf[pdf['case:concept:name']==id]
print(id)
min_los = 3
tlos = case_level.shape[0]

if id in offer_returned:
    outcome_check = 'O_Returned'
if id in offer_accepted:
    outcome_check = 'O_Accepted'
elif id in offer_cancelled:
    outcome_check = 'O_Cancelled'
elif id in offer_refused:
    outcome_check = 'O_Refused'

print(outcome_check)

min_idx = min(case_level.index)
max_idx = max(case_level.index)

# idx_outcome = case_level.loc[case_level['concept:name'] == outcome_check].index.tolist()[0]

idx = case_level.loc[case_level['concept:name'] == outcome_check].index.tolist()[0]

k_max = case_level.index.tolist().index(idx)


k = random.randint(min_los, k_max)
print(k)

trunc_case_level_1 = case_level.iloc[0:k,:]

trunc_case_level_2 = case_level.drop(index = case_level.index.tolist()[k:])

# for id in list(data[data['concept:name']=='O_Returned']['case:concept:name'].unique()):
for id in offer_returned:
    case_level = data.loc[data['case:concept:name']==id,:]
    idx = case_level.loc[case_level['concept:name'] == 'O_Returned'].index.tolist()[0]
    print(id, idx)

ids=pdf['case:concept:name'].unique()
pdf=pdf[pdf['case:concept:name'].isin(np.random.choice(ids,1000))]

In [12]:
import time, random

stime= time.time()

pdf.sort_values(by=[case_id_f,timestamp_f],ascending=True,inplace=True)

data = pdf.copy()
min_los = 7 #3, 5, 9
idx_to_drop = []

for id in case_ids:
    case_level = data.loc[data['case:concept:name']==id,:]
#     case_level.loc[:,timestamp_f]= case_level[timestamp_f].transform( lambda x: (x-x.min()/(x.max())))
#     data.loc[data['case:concept:name']==id,timestamp_f] = case_level.loc[:,timestamp_f]
#     print(case_level.head())
    tlos = case_level.shape[0]
    
    if id in offer_returned:
#         outcome_check = 'O_Returned'
        idx = case_level.loc[case_level['concept:name'] == 'O_Returned'].index.tolist()[0]
        k_max = case_level.index.tolist().index(idx)
    else:
        k_max = tlos
#     elif id in offer_accepted:
#         outcome_check = 'O_Accepted'
#     elif id in offer_cancelled:
#         outcome_check = 'O_Cancelled'
#     elif id in offer_refused:
#         outcome_check = 'O_Refused'

#     print(id, outcome_check)
#     idx = case_level.loc[case_level['concept:name'] == outcome_check,:].index.tolist()
#     print(idx)
#     if idx==[]:
#         print(case_level.loc[case_level['concept:name'] == outcome_check,:])
#     k_max = case_level.index.tolist().index(idx[0])
#     print(k_max)
    k = random.randint(min_los, k_max)
    idx_to_drop.extend(case_level.index.tolist()[k:])
#     print(idx_to_drop)
#     data[data['case:concept:name']==id,:]=case_level.drop(index = case_level.index.tolist()[k:], inplace=False)

print(time.time()-stime)

1610.284809589386


In [13]:
kpref_data = data.drop(index = idx_to_drop)

In [14]:
kpref_data.shape

(490325, 19)

In [15]:
print(pdf.shape)
print(kpref_data.shape)

(1202267, 19)
(490325, 19)


In [16]:
# data = pdf.copy()
kpref_data.columns

Index(['Action', 'org:resource', 'concept:name', 'EventOrigin', 'EventID',
       'lifecycle:transition', 'time:timestamp', 'case:LoanGoal',
       'case:ApplicationType', 'case:concept:name', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'Accepted', 'MonthlyCost',
       'Selected', 'CreditScore', 'OfferedAmount', 'OfferID'],
      dtype='object')

In [17]:
len(kpref_data['case:concept:name'].unique())

31509

In [18]:
case_id_f = 'case:concept:name'
data.groupby([case_id_f]).agg(count=('Action', 'count'))['count'].mean()

38.15630454790695

In [19]:
kpref_data=kpref_data.fillna(0)
# data.head()

In [20]:
def to_integer(dt_time):
    return 10000*dt_time.year + 100*dt_time.month + dt_time.day

In [21]:
#TIMESTAMP -  'time:timestamp'
#CUSTID - case:concept:name
# data = session_wrk_lkbk.copy()



categorical_cols = [ activity_f ,'case:LoanGoal', 'case:ApplicationType'] 

continuous_cols = [timestamp_f ,'case:RequestedAmount','FirstWithdrawalAmount', 'NumberOfTerms', 'CreditScore']

cols_to_drop = ['OfferedAmount','Action', 'org:resource', 'EventOrigin', 'EventID','lifecycle:transition', 'Accepted', 'MonthlyCost','Selected', 'OfferID']

# print(kpref_data.columns)
sub_data = kpref_data.drop(cols_to_drop, axis=1)
# sub_data = kpref_data
# sub_data['time'] = pd.to_datetime(sub_data[timestamp_f])

# sub_data.sort_values(by=[case_id_f,timestamp_f],ascending=True,inplace=True)

sub_data.loc[:,case_id_f] =  sub_data.loc[:,case_id_f].astype(str)

# sub_data = sub_data.drop([case_id_f], axis=1)

# sub_data.loc[:,timestamp_f]=(sub_data['time']).astype(int)/ 10**9

sub_data.loc[:,timestamp_f]=sub_data.loc[:,timestamp_f].apply(lambda x: to_integer(x))

sub_data['case:concept:name']=sub_data['case:concept:name'].apply(lambda x:x.split('_')[1])
case_flags['CaseID']=case_flags['CaseID'].apply(lambda x:x.split('_')[1])

# sub_data = sub_data.drop(['time'], axis=1)

for col in continuous_cols:
    scaler = MinMaxScaler(feature_range=(0, 1))
    sub_data.loc[:,col] =  scaler.fit_transform(sub_data[[col]])
    
# sub_data = sub_data.drop([timestamp_f], axis=1)

encoded_data = pd.get_dummies(sub_data, columns = categorical_cols) #TO TRY - binary encoding or other encodings


In [22]:
encoded_data.head(10)

,time:timestamp,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,CreditScore,concept:name_A_Accepted,concept:name_A_Cancelled,concept:name_A_Complete,concept:name_A_Concept,...,case:LoanGoal_Extra spending limit,case:LoanGoal_Home improvement,case:LoanGoal_Motorcycle,case:LoanGoal_Not speficied,"case:LoanGoal_Other, see explanation",case:LoanGoal_Remaining debt home,case:LoanGoal_Tax payments,case:LoanGoal_Unknown,case:ApplicationType_Limit raise,case:ApplicationType_New credit
686058,0.069505,1000086665,0.011111,0.000000,0.000000,0.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
686059,0.069505,1000086665,0.011111,0.000000,0.000000,0.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
686060,0.069505,1000086665,0.011111,0.000000,0.000000,0.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
686061,0.069505,1000086665,0.011111,0.000000,0.000000,0.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
686062,0.069505,1000086665,0.011111,0.000000,0.000000,0.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
686063,0.069505,1000086665,0.011111,0.000000,0.000000,0.0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
686064,0.069604,1000086665,0.011111,0.000000,0.000000,0.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
686065,0.069604,1000086665,0.011111,0.000000,0.000000,0.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
686066,0.069703,1000086665,0.011111,0.000000,0.000000,0.0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
686067,0.069703,1000086665,0.011111,0.066667,0.122222,0.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [23]:
encoded_data.columns

Index(['time:timestamp', 'case:concept:name', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'CreditScore',
       'concept:name_A_Accepted', 'concept:name_A_Cancelled',
       'concept:name_A_Complete', 'concept:name_A_Concept',
       'concept:name_A_Create Application', 'concept:name_A_Denied',
       'concept:name_A_Incomplete', 'concept:name_A_Submitted',
       'concept:name_A_Validating', 'concept:name_O_Cancelled',
       'concept:name_O_Create Offer', 'concept:name_O_Created',
       'concept:name_O_Refused', 'concept:name_O_Sent (mail and online)',
       'concept:name_O_Sent (online only)',
       'concept:name_W_Assess potential fraud',
       'concept:name_W_Call after offers',
       'concept:name_W_Call incomplete files',
       'concept:name_W_Complete application', 'concept:name_W_Handle leads',
       'concept:name_W_Shortened completion ',
       'concept:name_W_Validate application', 'case:LoanGoal_Boat',
       'case:LoanGoal_Business goal

In [24]:
sub_data.columns

Index(['concept:name', 'time:timestamp', 'case:LoanGoal',
       'case:ApplicationType', 'case:concept:name', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'CreditScore'],
      dtype='object')

In [25]:
case_flags.columns

Index(['CaseID', 'Flag'], dtype='object')

In [26]:
def class_balancer(y):
    sess_K1 = y[y.Flag == 1]['CaseID'].unique()
    sess_K0 = y[y.Flag == 0]['CaseID']
#     print(len(sess_K0))
#     print(len(sess_K1))
    smsk = np.random.rand(len(sess_K1)) > 0.51
    sess2keep = sess_K1[smsk]
#     print(len(sess2keep))
    fl = np.append(sess2keep,sess_K0)
#     sess2keep = sess2keep.append(sess_K1)
    
    return fl

In [27]:
len(encoded_data['case:concept:name'].unique())

31509

In [28]:
ids_to_keep = class_balancer(case_flags)
balanced_data = encoded_data[encoded_data[case_id_f].isin(ids_to_keep)]
balanced_target = case_flags[case_flags['CaseID'].isin(ids_to_keep)]

In [29]:
print(encoded_data.shape,balanced_data.shape)

(490325, 44) (317503, 44)


In [30]:
ids_to_keep

array(['1746793196', '1878239836', '619403287', ..., '2110228261',
       '2103458006', '1350494635'], dtype=object)

In [31]:
# balanced_target.CaseID.unique()

In [32]:
balanced_data=balanced_data.sort_values(case_id_f)
balanced_target=balanced_target.sort_values('CaseID')

In [33]:
cust_ids = balanced_data[case_id_f].unique()

msk = np.random.rand(len(cust_ids)) < 0.5
train_ids = cust_ids[msk]
val_test_ids = cust_ids[~msk]

train_data = balanced_data[balanced_data[case_id_f].isin(train_ids)]

msk = np.random.rand(len(val_test_ids)) < 0.5
validate_ids = val_test_ids[msk]
test_ids = val_test_ids[~msk]

validate_data = balanced_data[balanced_data[case_id_f].isin(validate_ids)]
test_data = balanced_data[balanced_data[case_id_f].isin(test_ids)]

case_flags

train_t = balanced_target[balanced_target['CaseID'].isin(train_ids)]
validate_t = balanced_target[balanced_target['CaseID'].isin(validate_ids)]
test_t = balanced_target[balanced_target['CaseID'].isin(test_ids)]

# print(len(train[train.Flag==1]['SessionID'].unique())/len(train['SessionID'].unique())) #Sessions with flag =1 
# print(len(validate[validate.Flag==1]['SessionID'].unique())/len(validate['SessionID'].unique())) #Sessions with flag =1 
# print(len(test[test.Flag==1]['SessionID'].unique())/len(test['SessionID'].unique())) #Sessions with flag =1 

# # train_data = train.drop(['Flag','SessionID'], axis=1)
# # validate_data = validate.drop(['Flag','SessionID'], axis=1)
# # test_data = test.drop(['Flag','SessionID'], axis=1)
# train_data = train.drop(['Flag'], axis=1)
# validate_data = validate.drop(['Flag'], axis=1)
# test_data = test.drop(['Flag'], axis=1)

In [34]:

def reshape_data(data, lookback, nsamples, case_id, timestamp):
#     data=data.sort_values([ case_id, timestamp])
    data=data.sort_values([ case_id])
    unique_cases = data[case_id].unique()
    rdf=[]
    j=len(data.columns)
    k=lookback
    x = np.empty((nsamples,j,k))
    i=0
    for id in unique_cases:
        sub_data = data[data[case_id] == id]
        sub_data = sub_data.drop(case_id, axis=1)
#         sub_data = sub_data.drop([ case_id, timestamp], axis=1)
#         sub_data.loc[:,timestamp]= sub_data[timestamp].transform( lambda x: (x-x.min()/(x.max())))
#         sub_data.loc[:,'TIMESTAMP']= sub_data['TIMESTAMP'].transform( lambda x: x-x.min())
        temp2d = np.transpose(keras.preprocessing.sequence.pad_sequences(np.transpose(sub_data.values), maxlen=lookback, dtype='float32', value=0., padding='pre'))
        rdf.append(temp2d)
        i += 1
    return np.stack(rdf)

In [35]:
balanced_data

,time:timestamp,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,CreditScore,concept:name_A_Accepted,concept:name_A_Cancelled,concept:name_A_Complete,concept:name_A_Concept,...,case:LoanGoal_Extra spending limit,case:LoanGoal_Home improvement,case:LoanGoal_Motorcycle,case:LoanGoal_Not speficied,"case:LoanGoal_Other, see explanation",case:LoanGoal_Remaining debt home,case:LoanGoal_Tax payments,case:LoanGoal_Unknown,case:ApplicationType_Limit raise,case:ApplicationType_New credit
686058,0.069505,1000086665,0.011111,0.0,0.000000,0.000000,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
686070,0.069703,1000086665,0.011111,0.0,0.000000,0.000000,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
686069,0.069703,1000086665,0.011111,0.0,0.000000,0.000000,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
686068,0.069703,1000086665,0.011111,0.0,0.000000,0.000000,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
686066,0.069703,1000086665,0.011111,0.0,0.000000,0.000000,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918267,0.089604,999993812,0.066667,0.0,0.000000,0.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
918266,0.089604,999993812,0.066667,0.0,0.000000,0.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
918265,0.089604,999993812,0.066667,0.0,0.000000,0.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
918274,0.090000,999993812,0.066667,0.4,0.566667,0.765939,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [36]:
# from FU import *

lookback = 10

train_nsamples = len(train_data[case_id_f].unique())
test_nsamples = len(test_data[case_id_f].unique())
validate_nsamples = len(validate_data[case_id_f].unique())

train_rsd = reshape_data(train_data, lookback, train_nsamples, case_id_f, timestamp_f)
test_rsd = reshape_data(test_data, lookback, test_nsamples, case_id_f, timestamp_f)
validate_rsd = reshape_data(validate_data, lookback, validate_nsamples, case_id_f, timestamp_f)

In [37]:
train_target = train_t.Flag.to_numpy()
validate_target = validate_t.Flag.to_numpy()
test_target = test_t.Flag.to_numpy()

In [38]:
train_target.sum()/len(train_target)

0.5249703908409001

In [39]:
np.unique(train_target)

array([0, 1])

In [40]:
print(train_target.shape, train_rsd.shape)

(10132,) (10132, 10, 43)


In [41]:
np.unique(validate_target)

array([0, 1])

In [42]:
import torch
import torch.nn.functional as F
from torch import nn

class Attention(nn.Module):
    def __init__(self, feature_dim, step_dim, bias=True, **kwargs):
        super(Attention, self).__init__(**kwargs)
        
        self.supports_masking = True

        self.bias = bias
        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.features_dim = 0
        
        weight = torch.zeros(feature_dim, 1, requires_grad=True)
        nn.init.kaiming_uniform_(weight)
        self.weight = nn.Parameter(weight)
        
        if bias:
            self.b = nn.Parameter(torch.zeros(step_dim, requires_grad=True))
        
    def forward(self, x, mask=None):
        feature_dim = self.feature_dim 
        step_dim = self.step_dim

        eij = torch.mm(
            x.contiguous().view(-1, feature_dim), 
            self.weight
        ).view(-1, step_dim)
        
        if self.bias:
            eij = eij + self.b
            
        eij = torch.tanh(eij)
        a = torch.exp(eij)
        
        if mask is not None:
            a = a * mask

        a = a / (torch.sum(a, 1, keepdim=True) + 1e-10)

        weighted_input = x * torch.unsqueeze(a, -1)
        
        return (torch.sum(weighted_input, 1), a)

In [43]:
from torch.utils.data import Dataset

class timeseries(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x,dtype=torch.float32)
        self.y = torch.tensor(y,dtype=torch.float32)
        self.len = x.shape[0]

    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]
  
    def __len__(self):
        return self.len

In [44]:
from sklearn import metrics

def validate_me(x,y):
    test_target = x
#     print(y)
    pred_test_y = (y.detach().numpy() > 0.).astype(int)

    matrix = metrics.confusion_matrix(test_target, pred_test_y ) 
    return("F1:",metrics.f1_score(test_target, pred_test_y ),"Accuracy:",metrics.accuracy_score(test_target, pred_test_y ),"ROC_AUC:",metrics.roc_auc_score(test_target, pred_test_y )) 

In [45]:
# train_rsd[0]

In [46]:
import torch
import torch.nn.functional as F
from torch import nn

number_of_features = train_rsd.shape[2]

hidden_layers = 64

class LSTMAttn_nn(nn.Module):
    def __init__(self):
        super(LSTMAttn_nn,self).__init__()
        self.bn = nn.BatchNorm1d(lookback)
        self.do = nn.Dropout(0.0)
        self.lstm = nn.LSTM(input_size=number_of_features,hidden_size=hidden_layers,num_layers=1,batch_first=False,bidirectional=True,dropout=0.0)
        self.fc1 = nn.Linear(in_features=hidden_layers,out_features=1)
        self.fc1 = nn.Linear(in_features=hidden_layers*2,out_features=1)
        self.attn = nn.Linear(hidden_layers , hidden_layers)
        self.attn_combine = nn.Linear(hidden_layers * 2, hidden_layers)
#         self.attention = Attention(hidden_layers,lookback)
        self.attention = Attention(hidden_layers* 2,lookback )

    def forward(self,x):
        output = self.do(x)
        bn_output = self.bn(output)
#         output = self.bn(x)
#         output = output[:,-1,:]
        lstm_output, _status = self.lstm(bn_output)

        lstm_attn, attn_wts  = self.attention(lstm_output)
        lstm_attn_do = self.do(lstm_attn)
#         relu_output = torch.tanh(self.fc1(lstm_attn_do))
#         relu_output = self.fc1(torch.relu(lstm_attn_do))
        relu_output = torch.relu(self.fc1(lstm_attn_do))
#         relu_output = torch.relu((lstm_attn_do)
        if self.training:
            return relu_output
        else:
            return(relu_output, attn_wts)

model = LSTMAttn_nn()

In [47]:

learning_rate=0.0005

epochs = 100
batch_size= 64

# class_weight = torch.tensor([1., 5.])


# criterion = torch.nn.CrossEntropyLoss()
# criterion = torch.nn.MSELoss()
criterion = torch.nn.BCEWithLogitsLoss(reduction='sum')
# optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
optimizer = torch.optim.Adagrad(model.parameters(),lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(),lr= 0.003, weight_decay=2e-2)
# train_pt = torch.tensor(train_rsd,dtype=torch.float32)
# target_pt = torch.tensor(train_target,dtype=torch.float32)

# dataset = timeseries(train_rsd[0:10000],train_target[0:10000])
# dataset = timeseries(train_rsd[0:500],train_target[0:500])
dataset = timeseries(train_rsd,train_target)

validate_set = timeseries(validate_rsd,validate_target)
# validate_set = timeseries(validate_rsd[0:1000],validate_target[0:1000])
# validate_set = timeseries(validate_rsd[0:1000],validate_target[0:1000])

#dataloader

from torch.utils.data import DataLoader 
train_loader = DataLoader(dataset,shuffle=True,batch_size=batch_size)
validate_loader = DataLoader(validate_set,shuffle=True,batch_size=batch_size)

from torch.autograd import Variable

for i in range(epochs):
    avg_loss = 0
    train_pred = torch.zeros(len(train_loader))
    train_tar = torch.zeros(len(train_loader))
    for j,data in enumerate(train_loader):
        optimizer.zero_grad()
        model.train()
#         print(data[0][0].view(-1,5,16))
        y_pred = model(data[:][0].view(-1,lookback,number_of_features)).reshape(-1)
#         print(y_pred.view(256,-1)[:10])
#         print(data[:][1].reshape(-1).view(256,-1) )
        loss = criterion(y_pred,data[:][1])
#         loss = criterion(y_pred.view(256,-1),data[:][1].view(256,-1).reshape(-1))
#         loss = criterion(y_pred.view(256,-1),data[:][1])
#         y_pred = model(train_pt).reshape(-1)
#         loss = criterion(Variable(y_pred.view(1,-1)),Variable(target_pt.view(1,-1)))
#         y_pred.backward()    
        loss.backward()
        clipping_value = 1.
        torch.nn.utils.clip_grad_norm_(model.parameters(), clipping_value)
        optimizer.step()
        
        avg_loss += loss.item() / len(train_loader)
        train_pred = torch.cat((train_pred, y_pred), 0)
        train_tar = torch.cat((train_tar, data[:][1]), 0)
#         train_pred.append(y_pred)
#         train_tar.append(data[:][1])
        
    if (i+1)%10 == 0 or i==0:
        print(i,"th epoch : ")
        
        print("Training metrics: Loss",avg_loss, validate_me(train_tar,train_pred))
        total_attn = np.zeros((1,lookback))
        avg_val_loss = 0
        c=0
        
        val_preds = torch.zeros(len(validate_loader))
        val_targets = torch.zeros(len(validate_loader))
        for k, val in enumerate(validate_loader):
            model.eval()
            model_out = model(val[:][0].view(-1,lookback,number_of_features))
#             print(model_out[0])
            attn = model_out[1]
            total_attn = np.vstack((total_attn,attn.detach().numpy()))
            c+=1
#             print(attn.shape)
            y_epoch = model_out[0].reshape(-1)
#             y_epoch = model_out.reshape(-1)
#             print(y_epoch,val[:][1])
            loss_epoch = criterion(y_epoch,val[:][1])
            
            avg_val_loss += loss_epoch.item() / len(validate_loader)
            val_preds = torch.cat((val_preds, y_epoch), 0)
            val_targets = torch.cat((val_targets, val[:][1]), 0)

            
        print("Validation metrics  : Loss", avg_val_loss, validate_me(val_targets,val_preds))
#         print("Runs: ",c)


0 th epoch : 
Training metrics: Loss 43.84368470030011 ('F1:', 0.685940594059406, 'Accuracy:', 0.5376542610047614, 'ROC_AUC:', 0.5223274590385232)
Validation metrics  : Loss 43.62277231098693 ('F1:', 0.6931601069927398, 'Accuracy:', 0.5406178489702517, 'ROC_AUC:', 0.5198134299591396)
9 th epoch : 
Training metrics: Loss 41.016992808887814 ('F1:', 0.7031114952463268, 'Accuracy:', 0.5994558351958021, 'ROC_AUC:', 0.5883521890664427)
Validation metrics  : Loss 40.849521095370065 ('F1:', 0.7054708269203861, 'Accuracy:', 0.5985888634630053, 'ROC_AUC:', 0.583795624352653)
19 th epoch : 
Training metrics: Loss 39.099836793335726 ('F1:', 0.7132250580046404, 'Accuracy:', 0.639685161791857, 'ROC_AUC:', 0.6317566755103254)
Validation metrics  : Loss 38.90146401487752 ('F1:', 0.7141785446361592, 'Accuracy:', 0.6367276887871853, 'ROC_AUC:', 0.6260470682090147)
29 th epoch : 
Training metrics: Loss 37.80135817497783 ('F1:', 0.7187061638749073, 'Accuracy:', 0.6687396754445633, 'ROC_AUC:', 0.6635045519

In [48]:
validate_target[0:10000].sum()/len(validate_target[0:10000])

0.5301181483633547

In [49]:
# bal_train_rsd.shape

In [50]:
bal_train_rsd=train_rsd
bal_train_target=train_target
bal_train_ids=train_ids

bal_validate_ids=validate_ids

bal_test_ids=test_ids

In [51]:
# Running predictions on train dataset to obtain attention weights

test_set = timeseries( bal_train_rsd, bal_train_target)
test_loader = DataLoader(test_set,shuffle=False,batch_size=batch_size)


test_total_attn = np.zeros((1,lookback))
avg_test_loss = 0
c=0

test_preds = torch.zeros(len(test_loader))
test_targets = torch.zeros(len(test_loader))
for k, val in enumerate(test_loader):
    model.eval()
    model_out = model(val[:][0].view(-1,lookback,number_of_features))
#             print(model_out[0])
    attn = model_out[1]
    test_total_attn = np.vstack((test_total_attn,attn.detach().numpy()))
    c+=1
#             print(attn.shape)
    y_epoch = model_out[0].reshape(-1)
    loss_epoch = criterion(y_epoch,val[:][1])
    avg_test_loss += loss_epoch.item() #/ len(validate_loader)
    test_preds = torch.cat((test_preds, y_epoch), 0)
    test_targets = torch.cat((test_targets, val[:][1]), 0)

In [52]:
attn_df = pd.DataFrame(data=total_attn[1:]) 

validate_pred_attn = pd.DataFrame({'CaseID':bal_validate_ids , 'Predicted_Flag': val_preds.detach().numpy()[:len(bal_validate_ids)]})
print(validate_pred_attn.shape)
validate_pred_attn = pd.concat([validate_pred_attn, attn_df],axis=1)
print(validate_pred_attn.shape)

(5163, 2)
(5163, 12)


In [53]:
sum(bal_validate_ids==1107933452)

0

In [54]:
validate_pred_attn[validate_pred_attn['CaseID']==1107933452]

,CaseID,Predicted_Flag,0,1,2,3,4,5,6,7,8,9


In [55]:
test_attn_df = pd.DataFrame(data=test_total_attn[1:len(bal_train_ids)+1]) 

test_pred_attn = pd.DataFrame({'CaseID':bal_train_ids , 'Predicted_Flag': test_preds.detach().numpy()[:len(bal_train_ids)]})

# test_attn_df = pd.DataFrame(data=train_total_attn[1:len(bal_train_ids)+1]) 

# test_pred_attn = pd.DataFrame({'SessionID':bal_train_ids , 'Predicted_Flag': train_pred.detach().numpy()[:len(bal_train_ids)]})

print(test_pred_attn.shape)



test_pred_attn = pd.concat([test_pred_attn, test_attn_df],axis=1)
print(test_pred_attn.shape)
print(len(test_pred_attn.CaseID.unique()))

(10132, 2)
(10132, 12)
10132


In [56]:
test_pred_attn.describe()
test_pred_attn['Adj_Predicted_Flag'] = (test_pred_attn['Predicted_Flag']>0.).astype(int)

In [57]:
adj_train_pred=(train_pred>0.).detach().numpy().astype(int)
print(sum((train_tar.detach().numpy()==adj_train_pred))/len(adj_train_pred))

0.7273345641822953


In [58]:
sub_data

,concept:name,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,CreditScore
686058,A_Create Application,0.069505,"Other, see explanation",New credit,1000086665,0.011111,0.0,0.0,0.0
686059,A_Submitted,0.069505,"Other, see explanation",New credit,1000086665,0.011111,0.0,0.0,0.0
686060,W_Handle leads,0.069505,"Other, see explanation",New credit,1000086665,0.011111,0.0,0.0,0.0
686061,W_Handle leads,0.069505,"Other, see explanation",New credit,1000086665,0.011111,0.0,0.0,0.0
686062,W_Complete application,0.069505,"Other, see explanation",New credit,1000086665,0.011111,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
918282,W_Call after offers,0.090396,Caravan / Camper,New credit,999993812,0.066667,0.0,0.0,0.0
918283,W_Call after offers,0.090396,Caravan / Camper,New credit,999993812,0.066667,0.0,0.0,0.0
918284,W_Call after offers,0.090891,Caravan / Camper,New credit,999993812,0.066667,0.0,0.0,0.0
918285,W_Validate application,0.090891,Caravan / Camper,New credit,999993812,0.066667,0.0,0.0,0.0


In [59]:
case_flags

,CaseID,Flag
0,652823628,1
1,1691306052,0
2,428409768,1
3,1746793196,1
4,828200680,0
...,...,...
31504,1107993134,1
31505,2103458006,0
31506,59622033,1
31507,303923658,1


In [60]:
comb_data=sub_data.join(case_flags.set_index('CaseID'), on='case:concept:name')

In [61]:
# comb_data['CaseID'] = comb_data['case:concept:name'].apply(lambda x:x.split('_')[1])

In [62]:
# comb_data

,concept:name,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,CreditScore,Flag
686058,A_Create Application,0.069505,"Other, see explanation",New credit,1000086665,0.011111,0.0,0.0,0.0,0
686059,A_Submitted,0.069505,"Other, see explanation",New credit,1000086665,0.011111,0.0,0.0,0.0,0
686060,W_Handle leads,0.069505,"Other, see explanation",New credit,1000086665,0.011111,0.0,0.0,0.0,0
686061,W_Handle leads,0.069505,"Other, see explanation",New credit,1000086665,0.011111,0.0,0.0,0.0,0
686062,W_Complete application,0.069505,"Other, see explanation",New credit,1000086665,0.011111,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
918282,W_Call after offers,0.090396,Caravan / Camper,New credit,999993812,0.066667,0.0,0.0,0.0,1
918283,W_Call after offers,0.090396,Caravan / Camper,New credit,999993812,0.066667,0.0,0.0,0.0,1
918284,W_Call after offers,0.090891,Caravan / Camper,New credit,999993812,0.066667,0.0,0.0,0.0,1
918285,W_Validate application,0.090891,Caravan / Camper,New credit,999993812,0.066667,0.0,0.0,0.0,1


In [63]:
# nd = comb_data[['case:concept:name','time:timestamp','concept:name','CaseID','Flag']]
nd = comb_data[['case:concept:name','time:timestamp','concept:name','Flag']]

np.random.seed(420)


nn_test = nd[nd['case:concept:name'].isin(np.random.choice(bal_train_ids,10000))]


nn_validate = nd[nd['case:concept:name'].isin(np.random.choice(bal_validate_ids,100))]


nn_eval = nd[nd['case:concept:name'].isin(np.random.choice(bal_test_ids,1000))]

In [64]:
# (np.random.choice(bal_test_ids,1000))

In [65]:
#pre-process eval set - WINDOWSSS
# min_los= lookback/3
min_los = 3
uni_ids = nn_eval['case:concept:name'].unique()
nn_eval_pp = pd.DataFrame()

for id in uni_ids:
    id_sub = nn_eval[nn_eval['case:concept:name']== id]
    los = id_sub.shape[0]
#     print(id_sub.shape)
    for i in np.arange(min_los,los-1,1):
        id_sub.loc[:,'case:concept:name'] = str(id)+'_'+str(int(i))
#         print(i)
        sub = id_sub.iloc[0:int(i),:]
        nn_eval_pp = nn_eval_pp.append(sub)
        
# nn_eval_pp

/home/mcs001/20200984/.conda/envs/hpc-ird/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [66]:
# nn_validate[nn_validate['case:concept:name']==1107933452]

In [67]:
import time, random

stime= time.time()

running_traces = nn_validate.copy()
# running_traces = nn_validate[nn_validate['Flag']==1]
# min_los = int(lookback/2)
min_los = 3
idx_to_drop = []

case_ids = running_traces['case:concept:name'].unique()

for id in case_ids:
    case_level = running_traces.loc[running_traces['case:concept:name']==id,:]
    k_max = case_level.shape[0]

    k = random.randint(min_los, k_max)  #random K prefix
#     print(case_level.index.tolist()[k:])
    idx_to_drop.extend(case_level.index.tolist()[k:])
    
running_traces=running_traces.drop(idx_to_drop)

print(time.time()-stime)
print(running_traces.shape, nn_test.shape)

0.04734182357788086
(859, 4) (98834, 4)


In [68]:
import time, random

stime= time.time()

candidate_pool = nn_test.copy()
# min_los = int(lookback/2)
min_los = 3
idx_to_drop = []

case_ids = candidate_pool['case:concept:name'].unique()

for id in case_ids:
    case_level = candidate_pool.loc[candidate_pool['case:concept:name']==id,:]
    tlos = case_level.shape[0]

    k = tlos-1
#     print(case_level.index.tolist()[k:])
    idx_to_drop.extend(case_level.index.tolist()[k:])  # n-1 prefix
    
candidate_pool=candidate_pool.drop(idx_to_drop)

print(time.time()-stime)
print(candidate_pool.shape, nn_validate.shape)

27.75372886657715
(92465, 4) (1469, 4)


In [69]:
import pm4py
from pm4py.objects.log.obj import EventLog

def convert_df_to_EventLogObj(df, case_id, activity_key, timestamp_key):
    log = pm4py.format_dataframe(df,
                                 case_id = case_id,
                                 activity_key= activity_key,
                                 timestamp_key= timestamp_key)

    log = pm4py.convert_to_event_log(log)
    
    trace = EventLog()
    for i in range(1,len(log)):
        trace.append(log[i])
    
    return trace


In [70]:
# running_traces[running_traces['case:concept:name']==1107933452]

In [71]:
running_trace_obj = convert_df_to_EventLogObj(running_traces, case_id = "case:concept:name",
                                     activity_key= "concept:name",
                                     timestamp_key= "time:timestamp")
candidate_pool_obj = convert_df_to_EventLogObj(candidate_pool, case_id = "case:concept:name",
                                     activity_key= "concept:name",
                                     timestamp_key= "time:timestamp")
eval_set_obj = convert_df_to_EventLogObj(nn_eval_pp, case_id = "case:concept:name",
                                     activity_key= "concept:name",
                                     timestamp_key= "time:timestamp")

In [72]:
# running_trace_obj
from pm4py.objects.log.util import log_regex

# form a mapping dictionary associating each activity of the two logs to an ASCII character
mapping = log_regex.form_encoding_dictio_from_two_logs(running_trace_obj, candidate_pool_obj, parameters={})
# encode the second log (against which we want to align each trace of the first log)
list_encodings = log_regex.get_encoded_log(candidate_pool_obj, mapping, parameters={})   #ASCII encoding for logs in search space for candidate selection
# # optimization: keep one item per variant
set_encodings = set(list_encodings) # set absrraction
list_encodings = list(set_encodings) #list abstraction

list_encodings = sorted(list_encodings, key=lambda x: len(x)) #sorting the list by length


In [73]:
cand_pool_seq = {}

for trace in candidate_pool_obj:
    cand_pool_seq[trace._get_attributes()['concept:name']] = log_regex.get_encoded_trace(trace, mapping, parameters={})

In [74]:
# cand_pool_seq

In [75]:
eval_set_seq = {}

for trace in eval_set_obj:
    eval_set_seq[trace._get_attributes()['concept:name']] = log_regex.get_encoded_trace(trace, mapping, parameters={})

In [76]:
import stringdist

def get_closest_traces(encoded_trace, list_encodings, dist_thresh):
#     dist_thresh = 0.4
    closest_trace = list()
    c=-1
    for i in range(0, len(list_encodings)):
    #     print(len(list_encodings[i]))
        dist = (stringdist.levenshtein_norm( encoded_trace, list_encodings[i]))
    #     dist = (stringdist.levenshtein( encoded_trace, list_encodings[i]))
    #     if c == -1:
    #         min_dist=dist
        if dist <= dist_thresh:
            min_dist=dist
            closest_trace.append(list_encodings[i])
    #         closest_trace.append(log_regex.get_encoded_log(list_encodings[i], mapping, parameters={}))
            c += 1
#         elif dist == min_dist:
#             closest_trace.append(list_encodings[i])
#             c += 1


#     print(min_dist,c)
    return(closest_trace)

In [77]:
# validate_pred_attn[validate_pred_attn['CaseID']=='1113584867'].iloc[:,2:].iloc[:,0]

In [78]:
# cand_pool_seq[str(103222410)]

In [79]:

from weighted_levenshtein import lev, osa, dam_lev

def get_attn_wts(rt, idc, cand_id):
#     print(idc)
    runn_attn_wts = validate_pred_attn.loc[validate_pred_attn['CaseID']==((idc)),:].iloc[:,2:]
    cand_attn_wts = test_pred_attn.loc[test_pred_attn['CaseID']==((cand_id)),:].iloc[:,2:]
#     print(runn_attn_wts)
#     print(cand_attn_wts)
    f = validate_pred_attn.loc[validate_pred_attn['CaseID']==(int(idc)),'Predicted_Flag']
    # print(f.any())
    pred_flag = int(f.any())
    # print(pred_flag)

#     sign_mul = 1 if pred_flag==0 else -1
#     print(sign_mul)
    n=9
#     print(cand_pool_seq[str(idc)])
    los = len(cand_pool_seq[str(cand_id)])

    substitute_costs = np.ones((128, 128), dtype=np.float64)
    if los<lookback:
        s_los=los
    else:
        s_los=lookback
        
    for i in np.arange(s_los,0,-1):
#         print(cand_id)
        curr_acc = cand_pool_seq[str(cand_id)][i-1]
#         print(i-1)
#         print(cand_attn_wts)
#         print(len(cand_attn_wts.loc[:,:]), i)
        attwt = (cand_attn_wts.iloc[0,i-1])
        if attwt.any() > float(1/lookback):
            curr_acc_cost = (attwt) - 1
    #         print(curr_acc_cost)
            substitute_costs[ord(curr_acc), :] = curr_acc_cost
    
    los = len(rt)
    if los<lookback:
        s_los=los
    else:
        s_los=lookback
    
    for i in np.arange(s_los,0,-1):
#         print(i)
        curr_acc = rt[i-1]
        attwt = (runn_attn_wts.iloc[:,i-1])
        if attwt.any() > float(1/lookback):
            curr_acc_cost = (attwt ) + 1
    #         print(curr_acc_cost)
            substitute_costs[:, ord(curr_acc)] = curr_acc_cost

    return substitute_costs
#     print(curr_acc_cost)


In [80]:
import stringdist

def get_closest_traces_attn(encoded_trace, trace_id, dist_thresh):
#     dist_thresh = 0.4
    closest_trace = list()
    c=-1
#     sub_costs = get_attn_wts(trace_id)
    word = encoded_trace
    for i in cand_pool_seq:
#         print(i)
        sub_costs = get_attn_wts(word, trace_id, i)
        word1 = cand_pool_seq[i]
        dist = lev(word,word1, substitute_costs=sub_costs)/max(len(word),len(word1))
    #     print(len(list_encodings[i]))
#         dist = (stringdist.levenshtein_norm( encoded_trace, list_encodings[i]))
        if dist <= dist_thresh:
            min_dist=dist
            closest_trace.append(i)
            c += 1
    return(closest_trace)

In [81]:
def get_eval_traces(encoded_trace, trace_id, dist_thresh):
#     dist_thresh = 0.4
    closest_trace = list()
    c=-1
#     sub_costs = get_attn_wts(trace_id)
    word = encoded_trace
    for i in eval_set_seq:
#         print(i)
#         sub_costs = get_attn_wts(word, trace_id, i)
        word1 = eval_set_seq[i]
        dist = lev(word,word1)/max(len(word),len(word1))
    #     print(len(list_encodings[i]))
#         dist = (stringdist.levenshtein_norm( encoded_trace, list_encodings[i]))
        if dist <= dist_thresh:
            min_dist=dist
            closest_trace.append(i)
            c += 1
    return(closest_trace)

In [82]:
st=time.time()

num_matches = np.zeros(len(running_trace_obj))
num_seqmatches = np.zeros(len(running_trace_obj))
set_of_close_trace = {}
e=0
i=0
ids_for_close_trace = {}

ids_for_Eval_trace = {}

for trace in running_trace_obj:
#     print(trace)
    encoded_trace = log_regex.get_encoded_trace(trace, mapping, parameters={}) #ASCII encoding for running trace
    trace_id = str(trace._get_attributes()['concept:name'])
#     print(type(trace_id))
    closest_trace_ids = get_closest_traces_attn(encoded_trace, trace_id, 0.5)
    
    eval_trace_ids = get_eval_traces(encoded_trace, trace_id, 0.5)
    ids_for_Eval_trace[trace_id] = eval_trace_ids
    if len(closest_trace_ids) == 0:
        e+=1
#         closest_traces = get_closest_traces(encoded_trace, list_encodings, 0.4)
    ids_for_close_trace[trace_id] = closest_trace_ids

    num_matches[i] = len(closest_trace_ids) 
    i+=1
    
print(time.time()-st)

print(e)

2544.026031970978
0


In [83]:
from pm4py.util import string_distance
import difflib
from pm4py.objects.log.util import log_regex

num_matches = np.zeros(len(running_trace_obj))
num_seqmatches = np.zeros(len(running_trace_obj))
set_of_close_trace = {}
e=0
i=0
for trace in running_trace_obj:
#     print(trace)
    encoded_trace = log_regex.get_encoded_trace(trace, mapping, parameters={}) #ASCII encoding for running trace
    closest_traces = get_closest_traces(encoded_trace, list_encodings, 0.5)
    if len(closest_traces) == 0:
        e+=1
#         closest_traces = get_closest_traces(encoded_trace, list_encodings, 0.4)
    set_of_close_trace[trace._get_attributes()['concept:name']] = closest_traces
#     if encoded_trace in set_encodings:  #if exact match is found
#         argmin_dist = encoded_trace
# #         print("Found")
#     else:
#         argmin_dist = string_distance.argmin_levenshtein(encoded_trace, list_encodings)  #returns closest string
    #     print(argmin_dist)
    c=0
#     for tr in closest_traces:
#         seq_match = difflib.SequenceMatcher(None, encoded_trace, tr).get_matching_blocks()  #finds matching sequences in closest string
#         c += (len(seq_match) - 1)
#     num_seqmatches[i] = c
    num_matches[i] = len(closest_traces) 
    i+=1
    
print(e)

0


In [84]:
st = time.time()

ids_for_close_trace0 = {}
for k in set_of_close_trace:
    ids_for_close_trace0[k]=[]
    for seq in set_of_close_trace[k]:
        for k2 in cand_pool_seq:
            if seq==cand_pool_seq[k2]:
                ids_for_close_trace0[k].append(k2.rstrip())
                
print(time.time()-st)

16.876777410507202


In [85]:
def pop_prefix(prefix, los, n):
    if len(prefix.split('-')) <= n:
        return prefix
    else:
        split_prefix = prefix.split('-')[1:]
        pref='-'.join(str(e) for e in split_prefix)
        return pref
            
def update_los(pref,los,n):
    if len(pref.split('-'))>n:
        return los
    else:
        return los+1

def create_transys_n(sess_page_level,n):
    transys = {}
    unique_sess =  sess_page_level['case:concept:name'].unique()
    sess_feat_row = 0
    los = 0
    for sess in unique_sess:
#         sess_feat.at[sess_feat_row,'CaseID'] = sess
        session_level = sess_page_level.loc[sess_page_level['case:concept:name']==sess]
#         print(session_level.shape[0])
        first_page=1
        c = 0
        prefix = "SOS"
        prev=""
        for index,row in session_level.iterrows():
            c +=1
            
            prefix = pop_prefix(prefix, los, n)
#             print(prefix)
            curr = session_level.at[index,'concept:name']
            if first_page==1:
                first_page=0
#                 prefix.append(curr)
#             if prev!=curr:
                prefix = prefix + '-' + curr
#                 print("1", prev, curr, prefix)
                los = 1
#                     los += 1
                prev=curr
#                 print(prefix)
                
#                 continue
            else:
#                 print(prev,curr)
                if prev!=curr:
                    if prefix in transys.keys() and curr not in transys.get(prefix):
                        
                        transys.setdefault(prefix, []).append(curr)
#                         print(str(transys))
#                 prefix.append(curr)
                        prefix = prefix + '-' + curr
#                         print("2", prev, curr, prefix)
#                         los = update_los(prefix,los,n)
                        los += 1
                    elif prefix not in transys.keys():
#                         prefix = pop_prefix(prefix, los, n)
                        transys.setdefault(prefix, []).append(curr)
                        prefix = prefix + '-' + curr
#                         print("3", prev, curr, prefix)
#                         los = update_los(prefix,los,n)
                        los += 1
#                         print(str(transys))
                    prev=curr
#                     print("Adding")
                else:
                    if prefix not in transys.keys():
                        transys.setdefault(prefix, []).append(curr)
                    elif curr not in transys.get(prefix):
                        transys.setdefault(prefix, []).append(curr)
#             if c==session_level.shape[0]:
#                 prefix = pop_prefix(prefix, los, n)
#                 if prefix in transys.keys() and "EOS" not in transys.get(prefix):
#                     transys.setdefault(prefix, []).append("EOS")
#                 elif prefix not in transys.keys():
#                     transys.setdefault(prefix, []).append("EOS")
#                     print("Skipping")
    
#     bigram_df = pd.DataFrame(vect.fit_transform(sess_feat['twograms'].values).todense(), columns = vect.get_feature_names())

#     final_df = pd.concat([sess_feat,bigram_df], axis=1)
    return(transys)

In [86]:
# create_transys_n(running_traces[running_traces['case:concept:name']==(id)].sort_values(['time:timestamp']),3)

In [87]:
from scipy import stats
from statsmodels.stats import weightstats as stests

c=0
e=0
npk = 0 #No key in positive outcomes subset
nnk = 0 #No key in negative outcomes subset
nra = 0 #No reccomended activities
knf = 0 #Running trace last state not found in eval TS
nevtr = 0 #No evaluation traces ffound

tot_cnt=0
pos_trc_cnt=0
pos_rec_cnt=0
neg_rec_cnt=0

valid_rec_cnt=0

ev_trc_cnt=0
pos_sub_cnt=0
neg_sub_cnt=0

pos_sub_imp=0
pos_sub_wrs=0
pos_sub_imp_sg=0
pos_sub_wrs_sg=0

results = {}

pot_recs_for_running_tr = {}

id_for_recofoll = {}
id_for_reconfoll = {}


st = time.time()

for id in ids_for_close_trace:
    tot_cnt+=1
    l1 = set(ids_for_close_trace[str(id)])
    l2 = set(ids_for_close_trace0[str(id)])
    if(len(l1)==0):
        e+=1
    if(l1==l2):
#         print(l1, l2)
        c+=1
#     print(id)
    if(len(l1)!=0):
        pos_trc_cnt+=1
        
#         print(running_traces[running_traces['case:concept:name']==id])
        #get state of running trace
#         print(create_transys_n(running_traces[running_traces['case:concept:name']==int(id)].sort_values(['time:timestamp']),3))
#         print(create_transys_n(running_traces[running_traces['case:concept:name']==int(id)].sort_values(['time:timestamp']),3).keys())
        rt_state = list(create_transys_n(running_traces[running_traces['case:concept:name']==(id)].sort_values(['time:timestamp']),2).keys())[-1]
        #get transition system from similar traces
#         print(list(l1))
#         closest_df = nn_test[nn_test['case:concept:name'].isin(list(map(int,l1)))]
        closest_df = nn_test[nn_test['case:concept:name'].isin(l1)]
#         print(closest_df)
        closest_df_pos = closest_df[closest_df['Flag']==0]
        closest_df_neg = closest_df[closest_df['Flag']==1]

        ts_closest_df_pos= create_transys_n(closest_df_pos.sort_values(['case:concept:name','time:timestamp']),2)
        ts_closest_df_neg= create_transys_n(closest_df_neg.sort_values(['case:concept:name','time:timestamp']),2)
        
#         pos_rec
#         neg_rec

#         print(rt_state)
        reco = []
        pos_rec=[]
        neg_rec=[]
    
#         print(ts_closest_df_pos)
#         print(ts_closest_df_neg)
        f0 = rt_state not in list(ts_closest_df_pos.keys())
        f1 =  rt_state not in list(ts_closest_df_neg.keys())
        f2 = 0
        if f0:
            npk += 1
#             print("No pos pot recos")
#             reco = []
        else:
            pos_rec_cnt+=1
            pos_rec = ts_closest_df_pos[rt_state]
#             print("Pos out potential recos :", pos_rec)
        if f1:
            nnk += 1
#             print("No neg pot recos")
            reco=pos_rec
            f2=1
        else:
            neg_rec_cnt+=1
            neg_rec = ts_closest_df_neg[rt_state]
#             print("Neg out potential recos :", neg_rec)
#         print(ts_closest_df_pos[rt_state], ts_closest_df_neg[rt_state])
        
#         if(len(neg_rec)==0):
#                 print("No neg pot recos")
#                 reco=pos_rec
#         else:
        if not f0 and not f1 and not f2:
            for acc in pos_rec:
                if acc not in neg_rec:
                    reco.append(acc)
                    
        if len(reco)>0:
            valid_rec_cnt+=1
            pot_recs_for_running_tr[id]=reco
#             print("Recommended activity", reco)
            
            pos_reco_foll=0
            pos_reco_nfoll=0
            
            
            
            tot = len(ids_for_Eval_trace[id])
            
            if tot>0:
                ev_trc_cnt+=1
#                 print(tot)
                eval_trace_ids = ids_for_Eval_trace[id]
                foll_ids=[]
                nfoll_ids=[]
                for eval_id in eval_trace_ids:
#                     print(id, eval_id)
                    df = nn_eval_pp.loc[nn_eval_pp['case:concept:name']==(eval_id),['case:concept:name','time:timestamp','concept:name']]
#                     print(df.shape)
                    eval_trace_ts = create_transys_n(df.sort_values(['case:concept:name','time:timestamp']),2)
                    if rt_state in eval_trace_ts.keys():
                        next_eve = eval_trace_ts[rt_state]
#                         print(next_eve, reco)
                        if len(set(next_eve).intersection(set(reco)))>0:
                            pos_reco_foll+=1
                            foll_ids.append(eval_id)
                        else:
                            pos_reco_nfoll+=1
                            nfoll_ids.append(eval_id)
                    
                    else: 
                        nfoll_ids.append(eval_id)
                        pos_reco_nfoll+=1
                        knf+=1
#                 tot = pos_reco_foll+pos_reco_nfoll
#                 if tot>0:
                id_for_recofoll[id]= foll_ids
                id_for_reconfoll[id]=  nfoll_ids  
#                 print("Eval results (Total -- Recco foll -- Recco not followed):", tot, pos_reco_foll/tot, pos_reco_nfoll/tot)
                results[id]=(tot, pos_reco_foll/tot, pos_reco_nfoll/tot)
                if pos_reco_foll!=0:
                    pos_sub_cnt+=1
                    a = nn_eval_pp[nn_eval_pp['case:concept:name'].isin(foll_ids)]['Flag']
#                     print(sum(nn_eval_pp[nn_eval_pp['SessionID'].isin(foll_ids)]['Flag'])/len((nn_eval_pp[nn_eval_pp['SessionID'].isin(foll_ids)]['Flag'])))
                if pos_reco_nfoll!=0:
                    neg_sub_cnt+=1
                    b = nn_eval_pp[nn_eval_pp['case:concept:name'].isin(nfoll_ids)]['Flag']
#                     print(sum(nn_eval_pp[nn_eval_pp['SessionID'].isin(nfoll_ids)]['Flag'])/len((nn_eval_pp[nn_eval_pp['SessionID'].isin(nfoll_ids)]['Flag'])))
                if pos_reco_foll!=0 and pos_reco_nfoll!=0:
                    
                    ztest ,pval1 = stests.ztest(a, x2=b, value=0,alternative='two-sided')
#                     print("P-value: ",float(pval1))
                    if(np.mean(a)/np.mean(b)>1):
                        pos_sub_imp+=1
                        if(pval1 < 0.05):
                            pos_sub_imp_sg+=1
                    else:
                        pos_sub_wrs+=1
                        if(pval1 < 0.05):
                            pos_sub_wrs_sg+=1
                        
#                         print("Stat sign.")
            else:
                nevtr+=1
#             eval_set_seq
        else:
            nra+=1
#             print("No reco!")
            
print(time.time()-st)
    
print(e, c, npk, nnk, nra)
print("Total running traces checked:", tot_cnt)
print("Similar traces found for :", pos_trc_cnt)
print("Pos outcome recco found for:", pos_rec_cnt)
print("Neg outcome recco found for:", neg_rec_cnt)

print("Valid recco found for:",valid_rec_cnt)

print("Eval traces found for", ev_trc_cnt)
print("Pos sub found for", pos_sub_cnt)
print("Neg sub found for", neg_sub_cnt)
print("Pos sub outcome improvement for", pos_sub_imp)
print("Stat sig imp for", pos_sub_imp_sg)
print("Pos sub outcome worsens for", pos_sub_wrs)
print("Stat sig wrse for", pos_sub_wrs_sg)


1626.633210659027
0 0 6 4 80
Total running traces checked: 98
Similar traces found for : 98
Pos outcome recco found for: 92
Neg outcome recco found for: 94
Valid recco found for: 18
Eval traces found for 18
Pos sub found for 8
Neg sub found for 18
Pos sub outcome improvement for 2
Stat sig imp for 2
Pos sub outcome worsens for 6
Stat sig wrse for 4


In [88]:
from scipy import stats
from statsmodels.stats import weightstats as stests

c=0
e=0
npk = 0 #No key in positive outcomes subset
nnk = 0 #No key in negative outcomes subset
nra = 0 #No reccomended activities
knf = 0 #Running trace last state not found in eval TS
nevtr = 0 #No evaluation traces ffound

tot_cnt=0
pos_trc_cnt=0
pos_rec_cnt=0
neg_rec_cnt=0

valid_rec_cnt=0

ev_trc_cnt=0
pos_sub_cnt=0
neg_sub_cnt=0

pos_sub_imp=0
pos_sub_wrs=0
pos_sub_imp_sg=0
pos_sub_wrs_sg=0

results = {}

pot_recs_for_running_tr = {}

id_for_recofoll = {}
id_for_reconfoll = {}


st = time.time()

for id in ids_for_close_trace:
    tot_cnt+=1
    l1 = set(ids_for_close_trace[str(id)])
    l2 = set(ids_for_close_trace0[str(id)])
    if(len(l1)==0):
        e+=1
    if(l1==l2):
#         print(l1, l2)
        c+=1
#     print(id)
    if(len(l2)!=0):
        pos_trc_cnt+=1
        
#         print(running_traces[running_traces['case:concept:name']==id])
        #get state of running trace
#         print(create_transys_n(running_traces[running_traces['case:concept:name']==int(id)].sort_values(['time:timestamp']),3))
#         print(create_transys_n(running_traces[running_traces['case:concept:name']==int(id)].sort_values(['time:timestamp']),3).keys())
        rt_state = list(create_transys_n(running_traces[running_traces['case:concept:name']==(id)].sort_values(['time:timestamp']),2).keys())[-1]
        #get transition system from similar traces
#         print(list(l1))
#         closest_df = nn_test[nn_test['case:concept:name'].isin(list(map(int,l1)))]
        closest_df = nn_test[nn_test['case:concept:name'].isin(l2)]
#         print(closest_df)
        closest_df_pos = closest_df[closest_df['Flag']==0]
        closest_df_neg = closest_df[closest_df['Flag']==1]

        ts_closest_df_pos= create_transys_n(closest_df_pos.sort_values(['case:concept:name','time:timestamp']),2)
        ts_closest_df_neg= create_transys_n(closest_df_neg.sort_values(['case:concept:name','time:timestamp']),2)
        
#         pos_rec
#         neg_rec

#         print(rt_state)
        reco = []
        pos_rec=[]
        neg_rec=[]
    
#         print(ts_closest_df_pos)
#         print(ts_closest_df_neg)
        f0 = rt_state not in list(ts_closest_df_pos.keys())
        f1 =  rt_state not in list(ts_closest_df_neg.keys())
        f2 = 0
        if f0:
            npk += 1
#             print("No pos pot recos")
#             reco = []
        else:
            pos_rec_cnt+=1
            pos_rec = ts_closest_df_pos[rt_state]
#             print("Pos out potential recos :", pos_rec)
        if f1:
            nnk += 1
#             print("No neg pot recos")
            reco=pos_rec
            f2=1
        else:
            neg_rec_cnt+=1
            neg_rec = ts_closest_df_neg[rt_state]
#             print("Neg out potential recos :", neg_rec)
#         print(ts_closest_df_pos[rt_state], ts_closest_df_neg[rt_state])
        
#         if(len(neg_rec)==0):
#                 print("No neg pot recos")
#                 reco=pos_rec
#         else:
        if not f0 and not f1 and not f2:
            for acc in pos_rec:
                if acc not in neg_rec:
                    reco.append(acc)
                    
        if len(reco)>0:
            valid_rec_cnt+=1
            pot_recs_for_running_tr[id]=reco
#             print("Recommended activity", reco)
            
            pos_reco_foll=0
            pos_reco_nfoll=0
            
            
            
            tot = len(ids_for_Eval_trace[id])
            
            if tot>0:
                ev_trc_cnt+=1
#                 print(tot)
                eval_trace_ids = ids_for_Eval_trace[id]
                foll_ids=[]
                nfoll_ids=[]
                for eval_id in eval_trace_ids:
#                     print(id, eval_id)
                    df = nn_eval_pp.loc[nn_eval_pp['case:concept:name']==(eval_id),['case:concept:name','time:timestamp','concept:name']]
#                     print(df.shape)
                    eval_trace_ts = create_transys_n(df.sort_values(['case:concept:name','time:timestamp']),2)
                    if rt_state in eval_trace_ts.keys():
                        next_eve = eval_trace_ts[rt_state]
#                         print(next_eve, reco)
                        if len(set(next_eve).intersection(set(reco)))>0:
                            pos_reco_foll+=1
                            foll_ids.append(eval_id)
                        else:
                            pos_reco_nfoll+=1
                            nfoll_ids.append(eval_id)
                    
                    else: 
                        nfoll_ids.append(eval_id)
                        pos_reco_nfoll+=1
                        knf+=1
#                 tot = pos_reco_foll+pos_reco_nfoll
#                 if tot>0:
                id_for_recofoll[id]= foll_ids
                id_for_reconfoll[id]=  nfoll_ids  
#                 print("Eval results (Total -- Recco foll -- Recco not followed):", tot, pos_reco_foll/tot, pos_reco_nfoll/tot)
                results[id]=(tot, pos_reco_foll/tot, pos_reco_nfoll/tot)
                if pos_reco_foll!=0:
                    pos_sub_cnt+=1
                    a = nn_eval_pp[nn_eval_pp['case:concept:name'].isin(foll_ids)]['Flag']
#                     print(sum(nn_eval_pp[nn_eval_pp['SessionID'].isin(foll_ids)]['Flag'])/len((nn_eval_pp[nn_eval_pp['SessionID'].isin(foll_ids)]['Flag'])))
                if pos_reco_nfoll!=0:
                    neg_sub_cnt+=1
                    b = nn_eval_pp[nn_eval_pp['case:concept:name'].isin(nfoll_ids)]['Flag']
#                     print(sum(nn_eval_pp[nn_eval_pp['SessionID'].isin(nfoll_ids)]['Flag'])/len((nn_eval_pp[nn_eval_pp['SessionID'].isin(nfoll_ids)]['Flag'])))
                if pos_reco_foll!=0 and pos_reco_nfoll!=0:
                    
                    ztest ,pval1 = stests.ztest(a, x2=b, value=0,alternative='two-sided')
#                     print("P-value: ",float(pval1))
                    if(np.mean(a)/np.mean(b)>1):
                        pos_sub_imp+=1
                        if(pval1 < 0.05):
                            pos_sub_imp_sg+=1
                    else:
                        pos_sub_wrs+=1
                        if(pval1 < 0.05):
                            pos_sub_wrs_sg+=1
                        
#                         print("Stat sign.")
            else:
                nevtr+=1
#             eval_set_seq
        else:
            nra+=1
#             print("No reco!")
            
print(time.time()-st)
    
print(e, c, npk, nnk, nra)
print("Total running traces checked:", tot_cnt)
print("Similar traces found for :", pos_trc_cnt)
print("Pos outcome recco found for:", pos_rec_cnt)
print("Neg outcome recco found for:", neg_rec_cnt)

print("Valid recco found for:",valid_rec_cnt)

print("Eval traces found for", ev_trc_cnt)
print("Pos sub found for", pos_sub_cnt)
print("Neg sub found for", neg_sub_cnt)
print("Pos sub outcome improvement for", pos_sub_imp)
print("Stat sig imp for", pos_sub_imp_sg)
print("Pos sub outcome worsens for", pos_sub_wrs)
print("Stat sig wrse for", pos_sub_wrs_sg)


1212.5653755664825
0 0 5 5 67
Total running traces checked: 98
Similar traces found for : 98
Pos outcome recco found for: 93
Neg outcome recco found for: 93
Valid recco found for: 31
Eval traces found for 31
Pos sub found for 9
Neg sub found for 31
Pos sub outcome improvement for 3
Stat sig imp for 3
Pos sub outcome worsens for 6
Stat sig wrse for 6
